In [33]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
import nest_asyncio
import asyncio
import pandas as pd

nest_asyncio.apply()

In [34]:
documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)

/Users/brendan/Desktop - Brendan’s MacBook Air/llm/lib/python3.10/site-packages/pypdf/_utils.py:234: RuntimeWarning: coroutine 'DatasetGenerator.agenerate_questions_from_nodes' was never awaited
  m = regex.search(name + tok)


In [35]:
# Query the data
query_engine = index.as_query_engine()
response = query_engine.query('What forecast did they provide for Q1 2024? for revenue and revenue growth rate on an F/X neutral basis?')
print(response)

They provided a forecast for Q1 2024 of 13% revenue growth on an F/X neutral basis.


## "I want to parse my data into smaller chunks"

In [36]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(chunk_size=200)
documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()
response = query_engine.query('What did they say about the strength of non-English programming?')
print(response)

The strength of non-English programming was mentioned, including titles like Lupin S3 from France, Elite S7, and the movie Nowhere from Spain.


## I want to retrieve more context when I query"

In [37]:
query_engine = index.as_query_engine(similarity_top_k=5)
response = query_engine.query('What did they say about the strength of non-English programming?')
print(response)

The strength of non-English programming was mentioned, including titles like Lupin S3 from France, Elite S7, and the movie Nowhere from Spain.


## Low level API

In [38]:
from llama_index import Response
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.response_synthesizers import (ResponseMode, get_response_synthesizer)

In [39]:
# build index
index =VectorStoreIndex.from_documents(documents)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(response_mode=ResponseMode.COMPACT)

# assemble query engine
query_engine= RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query('What outlook did NFLX provide for outlook for 2024?, start with revenue, then operating margin')
print(response)

NFLX provided an outlook for 2024 in their shareholder letter. They mentioned that they expect a high single-digit percentage year-over-year increase in content amortization. However, the specific revenue outlook for 2024 is not mentioned in the given context. Regarding the operating margin, NFLX increased their full-year 2024 operating margin forecast from 22%-23% to 24% based on FX rates as of January 1, 2024. They mentioned that this reflects the weakening of the US dollar against most other currencies since October and their stronger-than-forecasted Q4'23 performance.


In [40]:
# default
query_engine=RetrieverQueryEngine.from_args(retriever, response_mode='simple_summarize')
response = query_engine.query('What outlook did NFLX provide for outlook for 2024?, start with revenue, then operating margin')
print(response)
# compact
query_engine=RetrieverQueryEngine.from_args(retriever, response_mode='compact')
response = query_engine.query('What outlook did NFLX provide for outlook for 2024?, start with revenue, then operating margin')
print(response)

# tree summaraize
query_engine=RetrieverQueryEngine.from_args(retriever, response_mode='tree_summarize')
response = query_engine.query('What outlook did NFLX provide for outlook for 2024?, start with revenue, then operating margin')
print(response)

NFLX provided an outlook for 2024 in their shareholder letter. They mentioned that they expect a high single-digit percentage year-over-year increase in content amortization. However, the specific revenue outlook for 2024 is not mentioned in the given context. Regarding the operating margin, NFLX increased their full-year 2024 operating margin forecast from 22%-23% to 24% based on FX rates as of January 1, 2024.
NFLX provided an outlook for 2024 in their shareholder letter. They mentioned that they expect a high single-digit percentage year-over-year increase in content amortization. However, the specific revenue outlook for 2024 is not mentioned in the given context. Regarding the operating margin, NFLX increased their full-year 2024 operating margin forecast from 22%-23% to 24% based on FX rates as of January 1, 2024. They attributed this increase to the weakening of the US dollar against most other currencies since October and their stronger-than-forecasted Q4'23 performance.
NFLX p

In [41]:
from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    Response,
)
from llama_index.llms import OpenAI

In [42]:
reader = SimpleDirectoryReader("data")
documents = reader.load_data()
data_generator = DatasetGenerator.from_documents(documents)
eval_questions = data_generator.generate_questions_from_nodes()

/Users/brendan/Desktop - Brendan’s MacBook Air/llm/lib/python3.10/site-packages/llama_index/evaluation/dataset_generation.py:184: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
/Users/brendan/Desktop - Brendan’s MacBook Air/llm/lib/python3.10/site-packages/llama_index/evaluation/dataset_generation.py:279: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [43]:
eval_questions = data_generator.generate_questions_from_nodes()

/Users/brendan/Desktop - Brendan’s MacBook Air/llm/lib/python3.10/site-packages/llama_index/evaluation/dataset_generation.py:279: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [44]:
eval_questions

['What were the key financial objectives set by Netflix at the start of 2023?',
 "How did Netflix's revenue growth in 2023 compare to the previous year?",
 "What was Netflix's operating margin for FY23 and how did it compare to the target?",
 'How much free cash flow did Netflix generate in 2023?',
 'What are the big opportunities Netflix sees for 2024?',
 'How does Netflix plan to increase its value to members in 2024?',
 'What new revenue and profit pool does Netflix plan to tap into in 2024?',
 'How does Netflix plan to deepen its connection with fans in 2024?',
 "What is Netflix's north star and how does it contribute to the company's growth?",
 'What are the forecasted revenue and operating income for Q1 2024?',
 'What was the year-over-year growth in revenue in Q4?',
 "How did the company's operating income and operating margin in Q4 compare to the previous year?",
 'What factors contributed to the higher-than-anticipated membership growth in Q4?',
 "What was the company's EPS in

In [45]:
gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

In [46]:
evaluator_gpt4 = RelevancyEvaluator(service_context=service_context_gpt4)


In [47]:
# create vector index
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context_gpt4
)

In [51]:
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame({
        "Query": [query],
        "Response": [str(response)],
        "Source": [response.source_nodes[0].node.get_content()[:1000] + "..."],
        "Evaluation Result": [eval_result],
    })
    eval_df = eval_df.style.set_properties(**{
        "inline-size": "600px",
        "overflow-wrap": "break-word",
    }, subset=["Response", "Source"])
    display(eval_df)


In [55]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[2])
eval_result = evaluator_gpt4.evaluate_response(
    query=eval_questions[2], response=response_vector
)

In [56]:
display_eval_df(eval_questions[2], response_vector, eval_result)


,Query,Response,Source,Evaluation Result
0,What was Netflix's operating margin for FY23 and how did it compare to the target?,"Netflix's operating margin for FY23 was 21%, which exceeded their target of 20%.","January23,2024 Fellowshareholders, Summary: ●We’vejustendedourfirstyearwithTedandGregasco-CEOsand,undertheirleadership,Netflixachievedthekeyfinancialobjectiveswesetatthestartof2023.We’ve: ○Acceleratedourgrowth,exitingtheyearwith12%revenuegrowth,upfrom6%in2022; ○GrownourFY23operatingmarginto21%from18%in2022,aheadofour20%target; ○Increasedourfreecashflowto$6.9Bfor2023. ●TheyearhasshowntheneedforNetflixtobalanceconsistencyandcontinuousimprovementwithadaptability.In2024,weseebigopportunitiesto: ○Increaseourvaluetomembersbyfurtherimprovingthecore(seriesandfilm),whilebroadeningouroffering(games,liveandsports-adjacentprogramming); ○Tapintoasignificantnewlongtermrevenueandprofitpoolbyscalingouradsbusiness; ○Deepenourconnectionwithfansthroughourmarketing,consumerproductsandinnovativenewliveexperiences. ●Webelievethereisplentyofroomforgrowthaheadasstreamingexpands,andournorthstarremainsthesame:tothrillmemberswithourentertainment.IfwecancontinuetoimproveNetflixfasterthanthecompetition,we’llhavean...","query=""What was Netflix's operating margin for FY23 and how did it compare to the target?"" contexts=None response=""Netflix's operating margin for FY23 was 21%, which exceeded their target of 20%."" passing=True feedback='YES' score=1.0 pairwise_source=None invalid_result=False invalid_reason=None"
